<center>
    <h1> Traffic Monitoring System </h1>

In [ ]:
import cv2
import numpy as np
import time
import math

#Web camera
vid=cv2.VideoCapture('video.mp4')                                  

count_line_pos = 570
count_line_pos3 = 425
clp =[count_line_pos,count_line_pos3]
min_width_rect=50                                                  
min_ht_rect=50
no=0
detect=[]
offset=3                                                        #Alllowable error between pixel
counter=[0,0]

wb=cv2.bgsegm.createBackgroundSubtractorMOG()                      

def center_handle(x,y,w,h):
    '''Detects center of detected object'''
    x1=int(w/2)
    y1=int(h/2)
    cx=x+x1
    cy=y+y1
    return cx,cy
   
while True:
    ret,frame=vid.read()                                            
                                                                     
    frame1 = frame[0:600, 0:600]
    frame2 = frame[0:600, 600:1200]
    cv2.line(frame1,(25,count_line_pos),(550,count_line_pos),(255,127,0),3)       #Lower Down going
    cv2.line(frame2,(130,count_line_pos),(550,count_line_pos),(127,255,0),3)       #Up going
    cv2.line(frame1,(300,count_line_pos3),(590,count_line_pos3),(255,127,0),3)       #Upper Down going
    cv2.line(frame2,(90,count_line_pos3),(330,count_line_pos3),(127,255,0),3)       #Up going
      
    f = [frame1, frame2]
    counterShape1=counterShape2=0
    cs = [counterShape1, counterShape2]
    col = [(0,0,255),(255,255,0)]
   
    for i in range (0,2):
        grey=cv2.cvtColor(f[i],cv2.COLOR_BGR2GRAY)
        blur=cv2.GaussianBlur(grey,(3,3),5)

        #Applying on each frame
        bgsub= wb.apply(blur)                                        
        dilat=cv2.dilate(bgsub,np.ones((5,5)))                        
        kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))      
        close=cv2.morphologyEx(dilat,cv2.MORPH_CLOSE,kernel)      
        close=cv2.morphologyEx(close,cv2.MORPH_CLOSE,kernel)
        cs[i],h =cv2.findContours(close,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
       
        #For drawing rectangles around detected objects
        for (j,c) in enumerate(cs[i]):

            (x,y,w,h)=cv2.boundingRect(c)

            validate_counter=((w>=min_width_rect) and (h>=min_ht_rect))      
            if not validate_counter:                                        
                continue

            cv2.rectangle(f[i],(x,y),(x+w,y+h),col[i],2)
            cv2.putText(f[i],"VEHICLE "+str(no),(x,y-20),cv2.FONT_HERSHEY_TRIPLEX,0.5,(255,244,255),1)
           
            center=center_handle(x,y,w,h)
            detect.append(center)
            cv2.circle(f[i],center,4,col[i],-1)    
            #print(detect)# center points
               
            for (x,y) in detect:  
                   #Count Vehicles
                if y < (clp[i-1] + offset) and y > (clp[i-1] - offset):
                    counter[i]+=1
                    no+=1
                    cv2.line(f[i],(25,clp[i-1]),(650,clp[i-1]),(0,127,255),3)
                    #detect.remove((x,y))
                    print("Vehicle ",str(no),":",str(counter[i]))
                    vtime1=time.time()
                    p=x;
                if y < (clp[i] + offset) and y > (clp[i] - offset):
                    vtime2=time.time()
                    t=vtime2-vtime1
                    q=x;
                    dis=math.dist([p,425],[q,570])
                    speed=dis/t
                    maxs = 6000
                    mins = 150
                    newmax = 150
                    newmin = 40
                    s = (((speed - mins)/(maxs - mins)) * (newmax - newmin)) + newmin
                
            cv2.putText(f[i],"Speed: "+str(round(s,2))+"km/hr",(x,y-40),cv2.FONT_HERSHEY_TRIPLEX,0.5,(0,127,255),1)
            detect.remove((x,y))
            
    cv2.putText(frame1,"VEHICLE COUNTER DOWN :"+ str(counter[0]),(50,70),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),4)
    cv2.putText(frame2,"VEHICLE COUNTER UP :"+ str(counter[1]),(50,70),cv2.FONT_HERSHEY_SIMPLEX,1,(127,127,127),4)
           
    cv2.imshow("Footage",frame)                                      
   
    if cv2.waitKey(1)==13:                                            
        break

cv2.destroyAllWindows()                                                
vid.release()                                                         